In [ ]:
class Vacuum2DEnvironment:
    def __init__(self, grid):
        """
        Initialize environment with 2D grid
        grid: 2D list representing room states ('clean' or 'dirty')
        """
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0]) if self.rows > 0 else 0

    def print_grid(self, agent_pos=None):
        """
        Print the current grid state with symbols
        A = Agent position, D = Dirty room, . = Clean room
        """
        for r in range(self.rows):
            row_str = ''
            for c in range(self.cols):
                cell = self.grid[r][c]
                if agent_pos == (r, c):
                    row_str += 'A'  # Agent position
                elif cell == 'dirty':
                    row_str += 'D'  # Dirty room
                else:
                    row_str += '.'  # Clean room
            print(row_str)
        print()

class SimpleReflexAgent2D:
    """
    Simple Reflex Agent for 2D environment
    Makes decisions based only on current perception
    """
    def __init__(self):
        self.position = (0, 0)  # Starting at top-left corner

    def perceive(self, env):
        """Perceive current room state"""
        r, c = self.position
        return env.grid[r][c]

    def act(self, percept, env):
        """Act based on current perception only"""
        r, c = self.position
        
        # Always clean if room is dirty
        if percept == 'dirty':
            return 'suck'
        
        # Move right if possible
        if c < env.cols - 1:
            return 'right'
        # Move to next row if at end of current row
        elif r < env.rows - 1:
            return 'down'
        else:
            return 'stop'

    def run(self, env):
        """Execute agent with logging and visualization"""
        print("=" * 50)
        print("SIMPLE REFLEX AGENT 2D")
        print("=" * 50)
        print("Initial environment:")
        env.print_grid(self.position)
        
        actions = []
        step = 0
        
        while True:
            step += 1
            percept = self.perceive(env)
            action = self.act(percept, env)
            r, c = self.position
            
            print(f"Step {step}: Agent at ({r}, {c})")
            print(f"  Perceived: {percept}, Action: {action}")
            actions.append(((r, c), action))

            if action == 'suck':
                env.grid[r][c] = 'clean'
                print(f"  → Cleaned room at ({r}, {c})")
            elif action == 'right':
                self.position = (r, c + 1)
                print(f"  → Moved right to ({self.position[0]}, {self.position[1]})")
            elif action == 'down':
                self.position = (r + 1, 0)
                print(f"  → Moved down to ({self.position[0]}, {self.position[1]})")
            elif action == 'stop':
                print("  → Agent stopped")
                break

            env.print_grid(self.position)

        print("Final environment state:")
        env.print_grid()
        print(f"Total actions: {len(actions)}\n")
        return actions

class ModelBasedReflexAgent2D:
    """
    Model-Based Reflex Agent for 2D environment
    Maintains internal state to track room history
    """
    def __init__(self):
        self.position = (0, 0)
        self.state = {}  # Internal state memory

    def perceive(self, env):
        """Perceive current room state"""
        r, c = self.position
        return env.grid[r][c]

    def update_state(self, percept):
        """Update internal state with current perception"""
        self.state[self.position] = percept

    def act(self, percept, env):
        """Act based on current percept and internal state"""
        self.update_state(percept)
        r, c = self.position
        
        # Always clean if room is dirty
        if percept == 'dirty':
            return 'suck'

        # Look for unvisited rooms in row-major order
        for i in range(env.rows):
            for j in range(env.cols):
                if (i, j) not in self.state:
                    # Navigate to unvisited room
                    if j > c and i == r:
                        return 'right'
                    elif i > r:
                        return 'down'
                    elif j < c and i == r:
                        return 'left' if c > 0 else 'down'

        return 'stop'

    def run(self, env):
        """Execute model-based agent with state tracking"""
        print("=" * 50)
        print("MODEL-BASED REFLEX AGENT 2D")
        print("=" * 50)
        print("Initial environment:")
        env.print_grid(self.position)
        
        actions = []
        step = 0
        
        while True:
            step += 1
            percept = self.perceive(env)
            action = self.act(percept, env)
            r, c = self.position
            
            print(f"Step {step}: Agent at ({r}, {c})")
            print(f"  Perceived: {percept}, Action: {action}")
            print(f"  Internal state: {len(self.state)} rooms visited")
            actions.append(((r, c), action))

            if action == 'suck':
                env.grid[r][c] = 'clean'
                self.update_state('clean')
                print(f"  → Cleaned room at ({r}, {c})")
            elif action == 'right':
                self.position = (r, c + 1)
                print(f"  → Moved right to ({self.position[0]}, {self.position[1]})")
            elif action == 'down':
                self.position = (r + 1, 0)
                print(f"  → Moved down to ({self.position[0]}, {self.position[1]})")
            elif action == 'stop':
                print("  → Agent stopped")
                break

            env.print_grid(self.position)

        print("Final environment state:")
        env.print_grid()
        print(f"Total actions: {len(actions)}")
        print(f"Rooms visited: {len(self.state)}\n")
        return actions

class GoalBasedAgent2D:
    """
    Goal-Based Agent for 2D environment
    Explicit goal: clean all rooms in the environment
    """
    def __init__(self):
        self.position = (0, 0)
        self.goal = "Clean all rooms in the environment"

    def perceive(self, env):
        """Perceive current room state"""
        r, c = self.position
        return env.grid[r][c]

    def goal_test(self, env):
        """Test if goal is achieved - all rooms clean"""
        return all(env.grid[r][c] == 'clean' 
                  for r in range(env.rows) 
                  for c in range(env.cols))

    def act(self, percept, env):
        """Act to achieve the goal of cleaning all rooms"""
        r, c = self.position
        
        # Always clean dirty rooms first
        if percept == 'dirty':
            return 'suck'
            
        # Check if goal is achieved
        if self.goal_test(env):
            return 'stop'
            
        # Move systematically to find dirty rooms
        if c < env.cols - 1:
            return 'right'
        elif r < env.rows - 1:
            return 'down'
        else:
            return 'stop'

    def run(self, env):
        """Execute goal-based agent"""
        print("=" * 50)
        print("GOAL-BASED AGENT 2D")
        print("=" * 50)
        print(f"Goal: {self.goal}")
        print("Initial environment:")
        env.print_grid(self.position)
        
        actions = []
        step = 0
        
        while True:
            step += 1
            percept = self.perceive(env)
            goal_achieved = self.goal_test(env)
            action = self.act(percept, env)
            r, c = self.position
            
            print(f"Step {step}: Agent at ({r}, {c})")
            print(f"  Perceived: {percept}, Goal achieved: {goal_achieved}")
            print(f"  Action: {action}")
            actions.append(((r, c), action))

            if action == 'suck':
                env.grid[r][c] = 'clean'
                print(f"  → Cleaned room at ({r}, {c})")
            elif action == 'right':
                self.position = (r, c + 1)
                print(f"  → Moved right to ({self.position[0]}, {self.position[1]})")
            elif action == 'down':
                self.position = (r + 1, 0)
                print(f"  → Moved down to ({self.position[0]}, {self.position[1]})")
            elif action == 'stop':
                final_goal = self.goal_test(env)
                print(f"  → Agent stopped. Final goal achieved: {final_goal}")
                break

            env.print_grid(self.position)

        print("Final environment state:")
        env.print_grid()
        print(f"Total actions: {len(actions)}\n")
        return actions

class UtilityBasedAgent2D:
    """
    Utility-Based Agent for 2D environment
    Uses utility functions to make optimal decisions
    """
    def __init__(self):
        self.position = (0, 0)
        self.utilities = {
            'clean_room': 10,      # Utility for clean rooms
            'dirty_room': -5,      # Penalty for dirty rooms
            'movement': -1,        # Cost of movement
            'cleaning_action': 15, # Reward for cleaning
            'unnecessary_move': -3 # Penalty for inefficient moves
        }

    def perceive(self, env):
        """Perceive current room state"""
        r, c = self.position
        return env.grid[r][c]

    def calculate_utility(self, action, env):
        """Calculate utility for a given action"""
        r, c = self.position
        utility = 0
        
        if action == 'suck':
            utility += self.utilities['cleaning_action']
        elif action == 'right':
            utility += self.utilities['movement']
            # Bonus if moving toward dirty room
            if c + 1 < env.cols and env.grid[r][c + 1] == 'dirty':
                utility += 3
            else:
                utility += self.utilities['unnecessary_move'] // 2
        elif action == 'down':
            utility += self.utilities['movement']
            # Bonus if next row has dirty rooms
            if r + 1 < env.rows:
                next_row_dirty = any(env.grid[r + 1][col] == 'dirty' 
                                   for col in range(env.cols))
                if next_row_dirty:
                    utility += 3
                else:
                    utility += self.utilities['unnecessary_move'] // 2
                    
        return utility

    def act(self, percept, env):
        """Choose action that maximizes utility"""
        r, c = self.position
        
        # Always clean if room is dirty (highest utility)
        if percept == 'dirty':
            return 'suck'
            
        # Calculate utilities for possible moves
        possible_actions = []
        if c < env.cols - 1:
            possible_actions.append(('right', self.calculate_utility('right', env)))
        if r < env.rows - 1:
            possible_actions.append(('down', self.calculate_utility('down', env)))
            
        # Choose action with highest utility
        if possible_actions:
            best_action = max(possible_actions, key=lambda x: x[1])
            if best_action[1] > -3:  # Only act if utility is reasonable
                return best_action[0]
                
        return 'stop'

    def run(self, env):
        """Execute utility-based agent"""
        print("=" * 50)
        print("UTILITY-BASED AGENT 2D")
        print("=" * 50)
        print(f"Utility functions: {self.utilities}")
        print("Initial environment:")
        env.print_grid(self.position)
        
        actions = []
        step = 0
        total_utility = 0
        
        while True:
            step += 1
            percept = self.perceive(env)
            action = self.act(percept, env)
            utility = self.calculate_utility(action, env)
            total_utility += utility
            r, c = self.position
            
            print(f"Step {step}: Agent at ({r}, {c})")
            print(f"  Perceived: {percept}, Action: {action}")
            print(f"  Action utility: {utility}, Total utility: {total_utility}")
            actions.append(((r, c), action))

            if action == 'suck':
                env.grid[r][c] = 'clean'
                print(f"  → Cleaned room at ({r}, {c})")
            elif action == 'right':
                self.position = (r, c + 1)
                print(f"  → Moved right to ({self.position[0]}, {self.position[1]})")
            elif action == 'down':
                self.position = (r + 1, 0)
                print(f"  → Moved down to ({self.position[0]}, {self.position[1]})")
            elif action == 'stop':
                print("  → Agent stopped")
                break

            env.print_grid(self.position)

        print("Final environment state:")
        env.print_grid()
        print(f"Total actions: {len(actions)}")
        print(f"Final total utility: {total_utility}\n")
        return actions

def demonstrate_all_agents():
    """
    Demonstrate all four agent types on the same environment
    """
    # Create test environment
    initial_grid = [
        ['dirty', 'clean', 'dirty'],
        ['clean', 'dirty', 'clean'],
        ['dirty', 'clean', 'dirty']
    ]
    
    print("🤖 2D VACUUM CLEANER AGENTS DEMONSTRATION")
    print("=" * 60)
    print("Grid Legend: A = Agent, D = Dirty, . = Clean")
    print(f"Environment size: {len(initial_grid)}x{len(initial_grid[0])}")
    print("=" * 60)
    
    # Test all agents
    agents = [
        SimpleReflexAgent2D(),
        ModelBasedReflexAgent2D(),
        GoalBasedAgent2D(),
        UtilityBasedAgent2D()
    ]
    
    results = {}
    
    for agent in agents:
        env = Vacuum2DEnvironment([row[:] for row in initial_grid])  # Deep copy
        actions = agent.run(env)
        results[agent.__class__.__name__] = len(actions)
    
    # Performance summary
    print("=" * 60)
    print("PERFORMANCE SUMMARY")
    print("=" * 60)
    for agent_name, action_count in results.items():
        print(f"{agent_name}: {action_count} actions")
    
    return results

# Run the demonstration
if __name__ == "__main__":
    demonstrate_all_agents()


🤖 2D VACUUM CLEANER AGENTS DEMONSTRATION
Grid Legend: A = Agent, D = Dirty, . = Clean
Environment size: 3x3
SIMPLE REFLEX AGENT 2D
Initial environment:
A.D
.D.
D.D

Step 1: Agent at (0, 0)
  Perceived: dirty, Action: suck
  → Cleaned room at (0, 0)
A.D
.D.
D.D

Step 2: Agent at (0, 0)
  Perceived: clean, Action: right
  → Moved right to (0, 1)
.AD
.D.
D.D

Step 3: Agent at (0, 1)
  Perceived: clean, Action: right
  → Moved right to (0, 2)
..A
.D.
D.D

Step 4: Agent at (0, 2)
  Perceived: dirty, Action: suck
  → Cleaned room at (0, 2)
..A
.D.
D.D

Step 5: Agent at (0, 2)
  Perceived: clean, Action: down
  → Moved down to (1, 0)
...
AD.
D.D

Step 6: Agent at (1, 0)
  Perceived: clean, Action: right
  → Moved right to (1, 1)
...
.A.
D.D

Step 7: Agent at (1, 1)
  Perceived: dirty, Action: suck
  → Cleaned room at (1, 1)
...
.A.
D.D

Step 8: Agent at (1, 1)
  Perceived: clean, Action: right
  → Moved right to (1, 2)
...
..A
D.D

Step 9: Agent at (1, 2)
  Perceived: clean, Action: down
  → 